In [1]:
!date

jeudi 3 octobre 2019, 17:00:37 (UTC+0200)


# Starting full process

#### Complete the data acquisition and downloading to follow.
Full fasta reference of [Danio rerio](ftp://ftp.ensembl.org/pub/release-97/fasta/danio_rerio/cdna/Danio_rerio.GRCz11.cdna.all.fa.gz)

10x v2 chemistry barcode [whitelist]( https://github.com/bustools/getting_started/releases/download/getting_started/10xv2_whitelist.txt)

Transcripts to Genes (my code*)

Genetic data: 
    - Read 1 FASTQ
    - Read 2 FASTQ

Install [kallisto](https://pachterlab.github.io/kallisto/download)  
Install [bustools](https://bustools.github.io/download)

#### For installing stuff, use the following.

In [2]:
import sys, os 
import urllib.request
from IPython.display import display, Markdown


In [3]:
# Install using pip preferably! in the current Jupyter kernel
#As example install SCANPY the python package for processing later the gene matrix.
#!conda install --yes --prefix {sys.prefix} scanpy 
!{sys.executable} -m pip install scanpy


### Set location and create the files distribution, for files results, downloads, processing and notebooks...


In [4]:
os.getcwd()
#Move to a new location which we create for the analysis.

'/mnt/haus/marius/old_SingleCell10xKallistoJupyter/JupyterNotebooks'

In [5]:
#Set the path where we will work and create the folders tree.
os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter")
os.chdir("/mnt/haus/marius/SingleCell10xKallistoJupyter")


In [6]:
#Create the main directory, and add the subfolders desired in a list.
subfolders=["ensemblData","JupyterNotebooks","busOutputs","bashScripts","kallistoOutputs"]
for i in subfolders:
    print("Created file:" ,i)
    os.mkdir(os.path.join(os.getcwd())+"/"+i)
print("Files listed in the current directory: ",os.listdir())

Created file: ensemblData
Created file: JupyterNotebooks
Created file: busOutputs
Created file: bashScripts
Created file: kallistoOutputs
Files listed in the current directory:  ['JupyterNotebooks', 'ensemblData', 'busOutputs', 'bashScripts', 'kallistoOutputs']


#### Get data from ensembl for the following steps
    Download each file in a folder and name it in the ensemblData.

In [7]:
import urllib.request
print('\nBeginning file download with wget module')

ensembl_downloads={"cdna":"ftp://ftp.ensembl.org/pub/release-97/fasta/danio_rerio/cdna/Danio_rerio.GRCz11.cdna.all.fa.gz",
                   "gff":"ftp://ftp.ensembl.org/pub/release-97/gff3/danio_rerio/Danio_rerio.GRCz11.97.gff3.gz",
                   "gtf":"ftp://ftp.ensembl.org/pub/release-97/gtf/danio_rerio/Danio_rerio.GRCz11.97.gtf.gz"}


downloads_folder="/mnt/haus/marius/SingleCell10xKallistoJupyter/ensemblData/"

#Download all in their respective folders...:
for k,v in ensembl_downloads.items():
    os.mkdir(os.path.join(downloads_folder+k+"/"))
    urllib.request.urlretrieve(v,os.path.join(downloads_folder + k+"/"+os.path.basename(v)))
    print("→ Download completed for: "+ os.path.basename(v))
                  
print("\nDownload Done")



Beginning file download with wget module
→ Download completed for: Danio_rerio.GRCz11.cdna.all.fa.gz
→ Download completed for: Danio_rerio.GRCz11.97.gff3.gz
→ Download completed for: Danio_rerio.GRCz11.97.gtf.gz

Download Done


### Create the index using  ENMSEBL cdna last version of _Danio rerio_


In [8]:
idx=os.path.basename(ensembl_downloads["cdna"]).strip(".gz")+".idx"
idx=os.path.join("/mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/"+idx)
reference_cdna="/mnt/haus/marius/SingleCell10xKallistoJupyter/ensemblData/cdna/Danio_rerio.GRCz11.cdna.all.fa.gz"

In [9]:
from IPython.display import display, Markdown

display(Markdown("### Use `!kallisto index -i` followed by `path for the new index name` and `cdna fasta file`"))
print("Use the path for the index name: "+ idx + "\n\n" + "and the path for the reference fasta: "+ reference_cdna)

### Use `!kallisto index -i` followed by `path for the new index name` and `cdna fasta file`

Use the path for the index name: /mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx

and the path for the reference fasta: /mnt/haus/marius/SingleCell10xKallistoJupyter/ensemblData/cdna/Danio_rerio.GRCz11.cdna.all.fa.gz


In [10]:
!kallisto index -i /mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx /mnt/haus/marius/SingleCell10xKallistoJupyter/ensemblData/cdna/Danio_rerio.GRCz11.cdna.all.fa.gz
 


[build] loading fasta file /mnt/haus/marius/SingleCell10xKallistoJupyter/ensemblData/cdna/Danio_rerio.GRCz11.cdna.all.fa.gz
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 364 target sequences
[build] warning: replaced 9065 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 445371 contigs and contains 68180911 k-mers 



In [13]:
display(Markdown("### In this case create the two samples which are going to be used, for an easier data storage"))

### In this case create the two samples which are going to be used, for an easier data storage

# Control 

In [15]:
display(Markdown("#### Run kallisto for Control Sample\n `!kallisto bus -i` followed by `the index generated before` then `-o the output path and name` then `-x 10xv2 chemistry used` then `-t 6 threads wanted` and reads `R1` and `R2` files"))

print("In this case create the two samples which are going to be used, for an easier data storage")
os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control")

print("The index generated before: " + str("/mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx") +  
      "\nThe output path and name: " + str("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control") +  
      "\nThe chemistry used: " + str("-x *10xv2* or 10xv3")+  
      "\nThe number of threads: " + str("-t 7")+  
      "\nThe reads for R1 path: "+str("/mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R1.fastq.gz")+  
      "\nThe reads for R2 path: "+str("/mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R2.fastq.gz"))

#### Run kallisto for Control Sample
 `!kallisto bus -i` followed by `the index generated before` then `-o the output path and name` then `-x 10xv2 chemistry used` then `-t 6 threads wanted` and reads `R1` and `R2` files

In this case create the two samples which are going to be used, for an easier data storage
The index generated before: /mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx
The output path and name: /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control
The chemistry used: -x *10xv2* or 10xv3
The number of threads: -t 7
The reads for R1 path: /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R1.fastq.gz
The reads for R2 path: /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R2.fastq.gz


In [16]:
!kallisto bus -i /mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx -o /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control -x 10xv2 -t 7 /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R1.fastq.gz /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R2.fastq.gz  


[index] k-mer length: 31
[index] number of targets: 57,775
[index] number of k-mers: 68,180,911
[index] number of equivalence classes: 208,724
[quant] will process sample 1: /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R1.fastq.gz
                               /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Control/wt1aOE_L1_R2.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 497,902,159 reads, 434,150,057 reads pseudoaligned


# Experimental

In [17]:
display(Markdown("#### Run kallisto for Experimental Sample\n `!kallisto bus -i` followed by `the index generated before` then `-o the output path and name` then `-x 10xv2 chemistry used` then `-t 6 threads wanted` and reads `R1` and `R2` files"))

os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental")

print("The index generated before: " + str("/mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx") +  
      "\nThe output path and name: " + str("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental") +  
      "\nThe chemistry used: " + str("-x *10xv2* or 10xv3")+  
      "\nThe number of threads: " + str("-t 7")+  
      "\nThe reads for R1 path: "+str("/mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R1.fastq.gz")+  
      "\nThe reads for R2 path: "+str("/mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R2.fastq.gz"))

#### Run kallisto for Experimental Sample
 `!kallisto bus -i` followed by `the index generated before` then `-o the output path and name` then `-x 10xv2 chemistry used` then `-t 6 threads wanted` and reads `R1` and `R2` files

The index generated before: /mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx
The output path and name: /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental
The chemistry used: -x *10xv2* or 10xv3
The number of threads: -t 7
The reads for R1 path: /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R1.fastq.gz
The reads for R2 path: /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R2.fastq.gz


In [18]:
!kallisto bus -i /mnt/haus/marius/SingleCell10xKallistoJupyter/kallistoOutputs/Danio_rerio.GRCz11.cdna.all.fa.idx -o /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/ -x 10xv2 -t 7 /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R1.fastq.gz /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R2.fastq.gz


[index] k-mer length: 31
[index] number of targets: 57,775
[index] number of k-mers: 68,180,911
[index] number of equivalence classes: 208,724
[quant] will process sample 1: /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R1.fastq.gz
                               /mnt/haus/marius/ines_data/Raw-data/Results-15102018/Experimental/wt1aOE4OHT_L1_R2.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 451,131,247 reads, 387,175,364 reads pseudoaligned


## Files ready for BUS processing

### Start processing the BUS file with bustools
                 
  
    1. Correct the barcodes using bustools correct: fix the barcodes that are within one hamming distance of the barcodes in the whitelist using whitelist.txt
    2. Sort the busfile using bustools sort: organize the busfile by barcode, UMI, set, and multiplicity, and
    3. Count records in the BUS with bustools count: generate the UMI count matrix using transcripts_to_genes.txt.


In [19]:
os.getcwd()

'/mnt/haus/marius/SingleCell10xKallistoJupyter'

# Start correcting CONTROL samples
*double check for paths*

In [20]:
#from IPython.display import display, Markdown
display(Markdown("# Control"))

#Change directory and start with the control sample.
print("Move to the folder containing the control sample and proceed to create the gene matrix.")
os.chdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control")
os.listdir()

print("download the whitelist from 10x using v2 chemistry")
wtl_url="https://github.com/bustools/getting_started/releases/download/getting_started/10xv2_whitelist.txt"
urllib.request.urlretrieve(wtl_url,os.path.join("/mnt/haus/marius/SingleCell10xKallistoJupyter/"+os.path.basename(wtl_url)))
print("→ Download completed for: "+ os.path.basename(wtl_url))
os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/genecount")
os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/tmp")

# Control

Move to the folder containing the control sample and proceed to create the gene matrix.
download the whitelist from 10x using v2 chemistry
→ Download completed for: 10xv2_whitelist.txt


#### To make the Gene Count Matrix we need bustools

##### Create a new folder for data storing.

Run the command with  `!` in local PC shell
Commands used: `!bustools correct`  

1. Use the whitelist, 10xv2 chemistry  
`-w /mnt/haus/marius/SingleCell10xKallistoJupyter/10xv2_whitelist.txt`  

2. The original bus file from the sample.  
`-p /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/output.bus |`  

3. Continue the pipe with the temporal file used for the sorting step. Using the temporal path created before and teh number of threads for the step...  
`bustools sort -T /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/tmp/ -t6 -p - |`

4. Continue with the pipe for the counting step.Using in the order established:  the new path for the genecounts, the transcriptstogenes file generated with t2g.py, the matrix of the bus output and the transcripts of the original BUSfile.  

`bustools count -o /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/genecount/genes -g /mnt/haus/marius/SingleCell10xKallistoJupyter/transcripts_to_genes.txt -e /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/matrix.ec -t /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/transcripts.txt --genecounts -`

### Important

   5.Appreciate at the end the `--genecounts -`

In [26]:
!bustools correct -w /mnt/haus/marius/SingleCell10xKallistoJupyter/10xv2_whitelist.txt -p /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/output.bus |bustools sort -T /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/tmp/ -t6 -p - |bustools count -o /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/genecount/genes -g /mnt/haus/marius/SingleCell10xKallistoJupyter/transcripts_to_genes.txt -e /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/matrix.ec -t /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/control/transcripts.txt --genecounts -

Found 737280 barcodes in the whitelist
Number of hamming dist 1 barcodes = 20550336
Processed 434150057 bus records
In whitelist = 421195659
Corrected = 3393121
Uncorrected = 9561277
Read in 424588780 BUS records


# Start correcting EXPERIMENTAL samples
*double check for paths*

In [27]:
#from IPython.display import display, Markdown
display(Markdown("# Experimental"))

#Change directory and start with the control sample.
print("Move to the folder containing the control sample and proceed to create the gene matrix.")
os.chdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/")
os.listdir()
os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/genecount")
os.mkdir("/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/tmp")

# Experimental

Move to the folder containing the control sample and proceed to create the gene matrix.


FileExistsError: [Errno 17] File exists: '/mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/genecount'

#### To make the Gene Count Matrix we need bustools

##### Create a new folder for data storing.

Run the command with  `!` in local PC shell
Commands used: `!bustools correct`  

1. Use the whitelist, 10xv2 chemistry  
`-w /mnt/haus/marius/SingleCell10xKallistoJupyter/10xv2_whitelist.txt`  

2. The original bus file from the sample.  
`-p /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/output.bus |`  

3. Continue the pipe with the temporal file used for the sorting step. Using the temporal path created before and teh number of threads for the step...  
`bustools sort -T /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/tmp/ -t6 -p - |`

4. Continue with the pipe for the counting step.Using in the order established:  the new path for the genecounts, the transcriptstogenes file generated with t2g.py, the matrix of the bus output and the transcripts of the original BUSfile.  

`bustools count -o /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/genecount/genes -g /mnt/haus/marius/SingleCell10xKallistoJupyter/transcripts_to_genes.txt -e /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/matrix.ec -t /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/transcripts.txt --genecounts -`

### Important

   5.Appreciate at the end the `--genecounts -`

In [28]:
!bustools correct -w /mnt/haus/marius/SingleCell10xKallistoJupyter/10xv2_whitelist.txt -p /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/output.bus |bustools sort -T /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/tmp/ -t6 -p - |bustools count -o /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/genecount/genes -g /mnt/haus/marius/SingleCell10xKallistoJupyter/transcripts_to_genes.txt -e /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/matrix.ec -t /mnt/haus/marius/SingleCell10xKallistoJupyter/busOutputs/experimental/transcripts.txt --genecounts -

Found 737280 barcodes in the whitelist
Number of hamming dist 1 barcodes = 20550336
Processed 387175364 bus records
In whitelist = 375570014
Corrected = 2951535
Uncorrected = 8653815
Read in 378521549 BUS records


In [ ]:
display(Markdown("# Samples ready to start using Seurat or Scanpy"))

### Load the content in scanpy and proceed the analysis and clustering with scanpy

#### Use the next notebook
    Control
    Experimental

### 